# Анализ данных

In [ ]:
import catboost
import faiss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df_base = pd.read_csv('data/base.csv', index_col=0)

In [ ]:
df_base

In [ ]:
df_base.shape

In [ ]:
num_columns = df_base.shape[1]
num_rows = 15
num_cols = 5 

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20))

for i, ax in enumerate(axes.flat):
    if i < num_columns:
        df_base.iloc[:, i].hist(bins=100, ax=ax)
        ax.set_title(f'Столбец {i}')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
df_train = pd.read_csv("data/train.csv", index_col=0)

In [ ]:
df_train.shape

In [ ]:
df_train

In [ ]:
df_train_visualisation = df_train.copy().drop(['Target'], axis=1)

In [ ]:
num_columns = df_train_visualisation.shape[1]

fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 20))

for i, ax in enumerate(axes.flat):
    if i < num_columns:
        df_train_visualisation.iloc[:, i].hist(bins=100, ax=ax)
        ax.set_title(f'Столбец {i}')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
del df_train_visualisation
del num_columns
del num_rows
del num_cols
del fig
del axes
del ax
del i

# Обработка данных

In [ ]:
for df in [df_base, df_train]:
    for column in ['6', '21', '25', '33', '44', '59', '65', '70']:
        df.drop(column, axis=1, inplace=True)

In [ ]:
targets = df_train["Target"]
df_train.drop("Target", axis=1, inplace=True)

In [ ]:
base_index = {k: v for k, v in enumerate(df_base.index.to_list())}

In [ ]:
train_index = {k: v for k, v in enumerate(df_train.index.to_list())}

In [ ]:
scaler = StandardScaler()
columns_base = df_base.columns
df_base = scaler.fit_transform(df_base)
df_base = pd.DataFrame(df_base, columns=columns_base)

columns_train = df_train.columns
df_train = scaler.fit_transform(df_train)
df_train = pd.DataFrame(df_train, columns=columns_train)

df_base['id'] = base_index
df_train['id'] = base_index

df_base.set_index('id', inplace=True)
df_train.set_index('id', inplace=True)

del scaler
del columns_base
del columns_train

# Работа с Faiss

In [ ]:
dims = df_base.shape[1] # количество признаков
n_cells = 500 # количество ячеек в индексе

quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

del dims
del n_cells
del quantizer

In [ ]:
idx_l2.train(np.ascontiguousarray(df_base).astype('float32'))

idx_l2.add(np.ascontiguousarray(df_base).astype('float32'))

In [ ]:
idx_l2.nprobe = 100 # количество ячеек, которые будут просмотрены при поиске
r, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), 5)

In [ ]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])

print(100 * acc / len(idx), "%")

In [ ]:
k = 50

In [ ]:
r, idx = idx_l2.search(np.ascontiguousarray(df_train.values).astype('float32'), k)

70.87% является лучшим показателем Faiss. Для ранджирования будем использовать большее количество значений k у отдадим их в другие модели. Если модель покажет значение меньше 70.87%, то будем использовать Faiss. Если больше, то заказчик примет решение о эфективности зятрат времени / качества.

# Ранжирование

In [ ]:
idx = np.array([[base_index[_] for _ in el] for el in idx], dtype=object)

In [ ]:
cb_target = np.array([[1 if x == target else 0 for x in el] for el, target in zip(idx.tolist(), targets.values.tolist())])
cb_target = cb_target.reshape(-1, 1)

In [ ]:
candidate_features = df_base.loc[idx.flatten()].values
object_features = df_train.values

reshaped_dist = r.reshape(-1,1)

repeated_object = np.repeat(object_features, k, axis=0)

cb_features = np.hstack((reshaped_dist, candidate_features, repeated_object))
print('CB_features:', cb_features.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cb_features, cb_target, test_size=0.1)

In [ ]:
del acc
del base_index
del candidate_features
del cb_features
del cb_target
del column
del object_features
del reshaped_dist
del repeated_object
del df_base
del df_train
del targets
del df
del idx
del r

In [ ]:
cb = catboost.CatBoostClassifier(verbose=True, random_state=42)

In [ ]:
cb.fit(X_train, y_train)
cb.predict(X_test)

In [ ]:
print(cb.score(X_test, y_test))

# Выводы

1. Работа с сэмплом базовых данных без предобработки в колличестве 50 000 строк: **13.8%** точность на тестовой выборке
2. Кластеры 50, вся база: **8.681%, 14с.** 
3. Кластеры 50, вся база, n_porbe 10: **13.787%, 140с.**
4. Кластеры 50, вся база, n_porbe 10, нормализация: **14.374%, 154с.**
5. Вся база, нормализация: **14.396%, 655с.**
6. Кластеры 500, вся база, n_probe 100, нормализация: **14.391%, 93с.**
7. Кластеры 1700, вся база, n_probe 300, нормализация: **14.392%, 131с.**
8. Кластеры 500, вся база, n_probe 100, нормализация, удаление признайов без распределения: **17.044%, 148с.**
9. Кластеры 500, вся база, n_probe 100, нормализация, удаление признайов без распределения, удаление признаков с разбросанными показателями: **30.646%, 139с.**
10. Кластеры 500, вся база, n_probe 100, нормализация, удаление всех признаков без нормального распределения: **69.661%, 120с.**
11. Кластеры 500, вся база, n_probe 100, удаление всех признаков без нормального распределения: **69.601%, 120с.**
12. Кластеры 1, вся база, удаление всех признаков без нормального распределения: **69.729%, 570с.**
13. Кластеры 500, вся база, n_probe 100, StandartScaler, удаление всех признаков без нормального распределения: **70.87%, 128с.**
14. Кластеры 500, вся база, n_probe 100, нормализация, StandartScaler, удаление всех признаков без нормального распределения: **70.432%, 125с.**
15. Кластеры 500, вся база, n_probe 100, MinMax, StandartScaler, удаление всех признаков без нормального распределения: **70.659%, 123с.**
16. Кластеры 500, вся база, n_probe 100, MinMax, удаление всех признаков без нормального распределения: **70.659%, 125с.**
17. Кластеры 500, вся база, n_probe 100, StandartScaler: **65.393%, 147с.**